In [4]:
import json
from ast import parse
from ast2json import ast2json
import sys
import cfgcreator as cfg
import extractor
import re

gen={}
kill={}
dlist=[]
index=-1

def runForList(lst,index):
    for item in lst:
        nodeType=str(type(item));
        if nodeType=='<class \'dict\'>':
            runForDict(item,index)
        elif nodeType=='<class \'list\'>':
            runForList(item,index)

def runForDict(node,index):
    
    if node['_type']=='AugAssign':
        extractVariables(node,index,1) 

    if node['_type']=='Assign':
        extractVariables(node,index,2)
                
    for key,value in node.items():
        nodeType=str(type(value));
        if nodeType=='<class \'dict\'>':
            runForDict(value,index);

        elif nodeType=='<class \'list\'>':
            runForList(value,index);
            
            
def extractVariables(node,index,part):
    #print(node)
    global gen
    global kill
    s=""
    value=node['value']
    if part==2:
        targets=node['targets']
    if part==1:
        targets=node['target']
    #print('\n',value,'\n',targets,'\n')
    G=[]
    K=[]
    if value['_type']=='Call':
        p=""
        if len(value['args'])>0:
            for i in value['args']:
                if i['_type']=='Name':
                    G+=i['id'],
                elif i['_type']=='BinOp':
                    v1,v2=0,0
                    for k in i['left']:
                        if k=='id':
                            v2=1
                    if v2:
                        G+=i['left']['id'],
                    v1,v2=0,0
                    for k in i['right']:
                        if k=='id':
                            v2=1
                    if v2:
                        G+=i['right']['id'],
                            
    elif value['_type']=='Compare':
        w,x=0,0
        for k in value['left']:
            if k=='id':
                x=1
        l=str()
        n,m=0,0
        for i in value['comparators']:
            for j in i:
                if j=='id':
                    m=1
        if m:
            G+=value['comparators'][0]['id'],
        if x:
            G+=value['left']['id'],
            
    elif value['_type']=='BinOp':
        w,x,v,p,t,y=0,0,0,0,0,0
        m=""
        n=""
        for k in value['left']:
            if k=='id':
                x=1
                G+=value['left']['id'],
            if k=='func':
                y=1
                G+=value['left']['func']['id'],

        if y:
            for i in value['left']['args']:
                if i['_type']=='Name':
                    m+=i['id']
                    G+=i['id'],
                elif i['_type']=='BinOp':
                    v1,v2=0,0
                    for k in i['left']:
                        if k=='id':
                            v2=1
                    if v2:
                        G+=i['left']['id'],
                    v1,v2=0,0
                    for k in i['right']:
                        if k=='id':
                            v2=1
                    if v2:
                        G+=i['right']['id'],

        for k in value['right']:
            if k=='id':
                p=1
                G+=value['right']['id'],
            if k=='func':
                t=1
                G+=value['right']['func']['id'],
        
        if t:
            for i in value['right']['args']:
                if i['_type']=='Name':
                    G+=i['id'],
                elif i['_type']=='BinOp':
                    v1,v2=0,0
                    for k in i['left']:
                        if k=='id':
                            v2=1
                    if v2:
                        G+=i['left']['id'],
                    v1,v2=0,0
                    for k in i['right']:
                        if k=='id':
                            v2=1
                    if v2:
                        G+=i['right']['id'],
            
    elif value['_type']=='Name':
        G+=value['id'],
            
    if part==2:
        for target in targets:
            K+=target['id'],
    if part==1:
        K+=targets['id'],
    if index not in gen:
        gen[index]=set(G)
        kill[index]=set(K)
    else:
        gen[index]=set(G).union(set(gen[index])-set(K))
        kill[index]=set(K).union(set(kill[index])-set(G))
    #print(gen,'\n')
    #print(kill,'\n','\n')

def Reaching_Def(keysList,basicBlocks,edgeList):
    #print(dlist)
    for item in keysList:
        index=item
        block=basicBlocks[item]
        lst=block.split('\n')
        while("" in lst):
            lst.remove("")
        for line in lst:
            #print(line)
            if 'branch[' not in line and 'while[' not in line and '=' in line:
                dlist.append(item)
                code=ast2json(parse(line))
                #print(code,'\n')
                runForList(code["body"],index)
        index+=1
    
    #print('GEN',gen,'\n')
    #print('KILL',kill)
    IN={}
    Out={}
    for item in keysList:
        IN[item]=set()
        Out[item]=set()

    change=True
    while change:
        change=False
        for item in dlist:
            p=edgeList[item]
            for x in p:
                if len(Out[x])>0:
                    IN[item]=In[item].union(Out[x])
            prev=Out[item]
            Out[item]=set(gen[item]).union(set(IN[item])-set(kill[item]))
            if prev!=Out[item]:
                change=True
        #print(IN,'\n')
        #print(Out,'\n','\n')
    return IN,Out
def clear():
    global gen
    global kill
    global dlist
    global index
    gen={}
    kill={}
    dlist=[]
    index=-1
